# Aufgabe 1


 Berechnen Sie mithilfe des Algorithmus der dividierten Differenzen sowie des Horner-Schemas
den Wert sin(62◦) aus den Datenpunkten


(50◦, sin 50◦), (55◦, sin 55◦), (60◦, sin 60◦), (70◦, sin 70◦).
Speichern Sie hierbei neben der oberen Diagonale [y0, δy0, . . . , δn y0] auch die untere Diagonale
[δ0yn = yn , δ1yn−1, . . . , δn y0]






In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# Calculate all the values for x and y and convert from angles to radiants


x_degree = np.array([50,55,60,70])

x = (np.pi / 180) * x_degree

y = np.sin(x)

In [3]:
def divided_diff_diagonal(x, y):
    '''
    Function to calculate the divided differences table
    and extract the upper and lower diagonals
    '''
    n = len(y)
    coef = np.zeros([n, n])
    
    #
    coef[:, 0] = y
    
    # Lists to store the diagonals
    upper_diagonal = [y[0]]  # Starts with y0
    lower_diagonal = [y[-1]]  # Starts with yn
    
    for j in range(1, n):
        for i in range(n - j):
            coef[i][j] = (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j] - x[i])
        
        
        # We then 
        upper_diagonal.append(coef[0][j])  # Upper diagonal: elements from the top
        lower_diagonal.append(coef[n-j-1][j])  # Lower diagonal: elements from the bottom
    
    return coef, upper_diagonal, lower_diagonal

# This is then t
print("Full dividing differences table")
print(divided_diff_diagonal(x,y)[0])
print("Top Row Elements")
print(divided_diff_diagonal(x,y)[1])
print("Lower diagonal Elements")
print(divided_diff_diagonal(x,y)[2])



Full dividing differences table
[[ 0.76604444  0.60856828 -0.40931616 -0.08631976]
 [ 0.81915204  0.53712913 -0.43944744  0.        ]
 [ 0.8660254   0.42208206  0.          0.        ]
 [ 0.93969262  0.          0.          0.        ]]
Top Row Elements
[0.766044443118978, 0.6085682814211631, -0.40931616313550856, -0.08631976192962809]
Lower diagonal Elements
[0.9396926207859083, 0.4220820622658598, -0.43944744423970183, -0.08631976192962809]


In [4]:

def horner(coef, x_data, x):
    """
    Implementation of the Horneer scheme for polynomial evaluation
    Here coeff is our dividing differences table we get,


    Short explanation:
    we start with len(coeff) - 2 because the polynomial gets evaluated at the highest degree
    -1 is the stop condition meaning we stop at i=0 with our polynomial evaluation
    the last -1 tells the loop to go down -1 with each iteration
    """
    result = coef[-1]
    for i in range(len(coef) - 2, -1, -1):
        result = result * (x - x_data[i]) + coef[i]
    return result





In [5]:
# Now we extract both slices from this dividing differences table for the horner scheme

_, div_top, lower_diag = divided_diff_diagonal(x,y)



sin_approx_v1 = horner(div_top,x, 62*np.pi/180)
print("-------------------------------------------------")
print("Approximated with 4 data points:", sin_approx_v1)
print("Actual sin value:", np.sin(62*np.pi/180))
print("Error:", (np.sin(62*np.pi/180)) - sin_approx_v1 )
print("-------------------------------------------------")


-------------------------------------------------
Approximated with 4 data points: 0.8829520604036581
Actual sin value: 0.8829475928589269
Error: -4.467544731268092e-06
-------------------------------------------------


### Small Note here:

I used the numpy insert method, this is somewhat a debatable approach when combining it with jupyter type scripts, because the insert goes of each time you run the cell, this can mess things up

In [6]:

# Add new data point (65°, sin(65°))
new_x = 65
new_y = np.sin(np.radians(new_x))

# Append the new data point
x = np.array([50,55,60,70,65]) * np.pi/180

lower_diag = np.array(lower_diag)
 

# Insert the new value
lower_diag = np.insert(lower_diag,0, np.sin(65*np.pi/180)) 


# Now do the dividing differences with this additional value

n = len(lower_diag)
k = len(x) - 1

for j in range(n-1):
    lower_diag[j +1] = (lower_diag[j] - lower_diag[j+1])/(x[k]-x[k-(j+1)])




div_top = np.append(div_top, lower_diag[n-1])

sin_approx_v2 = horner(div_top,x, 62 * np.pi/180)

print("-------------------------------------------------")
print("Approximated with 5 data points:", sin_approx_v2)
print("Actual sin value:", np.sin(62*np.pi/180))
print("Error:", (np.sin(62*np.pi/180)) - sin_approx_v2 )
print("-------------------------------------------------")


-------------------------------------------------
Approximated with 5 data points: 0.882947565950376
Actual sin value: 0.8829475928589269
Error: 2.690855083198329e-08
-------------------------------------------------


## Aufgabe 2

Berechnen Sie für verschiedene n die Lebesgue’schen Konstanten für äquidistante Stützstellen

$x_k = −1 + 2k$

n und Chebyshev-Punkte. Verwenden Sie den Algorithmus aus Aufgabe 5, um das
Maximum auf $[xk ,xk+1]$ zu bestimmen

## Die Lebesqueschen Konstanten

$\Lambda_n = \max_{\alpha \leq x \leq \beta} \sum_{i=0}^n |l_i(x)|$

Man hat hierbei Stützstellen $x_0 ... x_n$

Gegeben haben wir zei Typen von Stützstellen:

+ Äquidistant $x_k = -1 + \frac{2k}{n}$
+ Chebyschev Punkte $x_k = cos(\frac{(2k+1)\pi}{2(n+1)})$

In [7]:
# Define the golden ratio constant

gamma = (np.sqrt(5)-1)/2

# Next define our two types of points

def equi_points(n):
    return np.linspace(-1,1,n+1)

def cheby_points(n):
    return np.cos((2*np.arange(n+1)+ 1)* np.pi / (2*(n+1)))


# next we need to define our lagrangian basis polynomials

def lagrange_basis(x,k,xk):
    """
    Function that calculates a k-th lagrange basis polynomial on a point x
    """
    n = len(xk)-1
    Lk =1
    for j in range(n+1):
        if j != k:
            Lk *= (x-xk[j]) / (xk[k]-xk[j])
    return Lk

# Then we define our lebesque function

def lebesque(x,xk):
    n = len(xk)-1
    return sum(abs(lagrange_basis(x,k,xk)) for k in range(n+1))

# Lastly we define our golden section search algorithm from 5)

def golden_section_search(f,a,b,tol=1e-8):
    """ 
    Carries out the golden section search algorithm using a pre defined tolerance
    """
    A = b-gamma * (b-a)
    B = a+gamma *(b-a)

    while (b-a) > tol:
        if f(A) < f(B):
            a = A
            A = B
            B = a + gamma*(b-a)
        else:
            b = B
            B = A
            A = b-gamma*(b-a)
    return (a+b)/2 # approximation of maximum as a return

In [8]:
# Calculation of the Lebesque Consant

def lebesque_constant(xk):
    max_lebesque = 0 # initialize the constant
    for i in range(len(xk)-1):
        
        f = lambda x: lebesque(x ,xk)
        max_val = golden_section_search(f,xk[i],xk[i+1])
        max_lebesque = max(max_lebesque, lebesque(max_val,xk))
    return max_lebesque

# Then lets compute

for n in range(1,10):
    print("----------------------------------------------------")
    equidistant_points = equi_points(n)
    lebesque_equidistance = lebesque_constant(equidistant_points)
    print(f"Lebesque constant for the equidistant points: {lebesque_equidistance}")

    chebychev_points = cheby_points(n)
    lebesque_cheby = lebesque_constant(chebychev_points)
    print(f"Lebesque constant for cheby points: {lebesque_cheby}")
    print("-------------------------------------------------")
 

----------------------------------------------------
Lebesque constant for the equidistant points: 1.0
Lebesque constant for cheby points: 1.0
-------------------------------------------------
----------------------------------------------------
Lebesque constant for the equidistant points: 1.25
Lebesque constant for cheby points: 1.2500000000000002
-------------------------------------------------
----------------------------------------------------
Lebesque constant for the equidistant points: 1.6311303094408989
Lebesque constant for cheby points: 1.4267766952966368
-------------------------------------------------
----------------------------------------------------
Lebesque constant for the equidistant points: 2.207824397325843
Lebesque constant for cheby points: 1.561169610234237
-------------------------------------------------
----------------------------------------------------
Lebesque constant for the equidistant points: 3.106301159367827
Lebesque constant for cheby points: 1

# Pseudocode für Aufgabe 3

Wir betrachten also die Neville Interpolation

nun wollen wir einen Algorithmus mit wlechem der Wert $P_[0,n](x_s)$ des gesuchten Interpolationspolynoms $P_[0,n]$ an einer Stelle $x_s$ mit der obigen Rekursion berechnet

In [9]:
function Rekursive_Interpolation (x,y,xs,n):
    # x: Array of x-Values of Nodes
    # y: Array of y-values of Nodes
    # xs: The value we want to interpolate
    # n: the degree of our interpolation polynomial

    # Initialize the vector
    P = [0]*[n+1] # this is a vector with n+1 components
    
    # First initialize all the y values
    for i in range(n+1):
        P[i] = y[i]

    # now calculate the interpolation
    for m in range(1, n+1):
        for k in range(n-m,-1,-1):
            P[k] = ((xs - x[k])*P[k+1]-(xs - x[k+m])*P[k])/(x[k+m]-x[k])
    return P[0] # final value

SyntaxError: invalid syntax (1125004056.py, line 1)